In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
dataset_path = '../data'

train_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

# val_dataset = torchvision.datasets.ImageFolder(root='data/val', transform=transform)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

classes = train_dataset.classes

In [9]:
net = models.resnet18(pretrained=True)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, len(classes))  # クラス数に合わせて変更

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):  # データセットを複数回ループ

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(loss.item())        
# if i % 2000 == 1999:    # 2000ミニバッチごとに出力
        # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            # running_loss = 0.0

print('Finished Training')

0.6463813185691833
0.60259610414505
0.7401119470596313
0.25503453612327576
0.2345971167087555
0.219137504696846
0.9358153939247131
0.12778103351593018
0.12069139629602432
0.3338925242424011
0.592511773109436
0.4979628026485443
0.3985828757286072
0.6680927276611328
0.5507575869560242
0.45060503482818604
1.3238495588302612
0.7242766618728638
0.7983443140983582
0.6832973957061768
0.5568404197692871
0.6028366088867188
0.5915053486824036
0.23566165566444397
0.7986000180244446
0.3492504060268402
0.47228512167930603
0.553127646446228
0.04745672270655632
0.5859954357147217
0.19221268594264984
0.5088924765586853
0.3269500434398651
0.7922422885894775
0.23218069970607758
0.9879840016365051
0.34915691614151
0.4027463495731354
0.8498414754867554
0.8975763916969299
0.5388154983520508
0.3713487982749939
0.07956945896148682


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the validation images: {100 * correct / total}%')